In [ ]:
!pip install sentence_transformers

In [2]:
# https://huggingface.co/sentence-transformers/all-mpnet-base-v2

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
#Read health claims dataset as a list

import csv

with open("dataset/health_claims_dataset3.csv", newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    data.pop(0)

    
#improve the list format (avoid nested lists)
sentences1=[]
for i in range(len(data)):
    sentences1.append(data[i][0])

#Compute embeddings for our health claims dataset
embeddings1 = model.encode(sentences1, convert_to_tensor=True)


In [5]:
#function to split caption into sentences

# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    if (text.find('.')==-1):
        text = text + "."
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")    
    sentences = [s.strip() for s in sentences]
    for i in range(len(sentences)):
        if sentences[i]=='': sentences.pop(i)
    return sentences

In [6]:
#Breaking up an instagram caption into separate setences

# caption = "food with a 'good source of calcium' helps reduce weight significantly. calcium for healthy bones and teeth. Diets high in calcium may reduce the risk of osteoporosis in people 65 years and over. "
# caption = "eating an apple a day is going to help you lose your tension"
# caption = "Cannabis use can help you quit your addiction to smoking"
caption = "regular cannabis use can greatly contribute to stopping addictive habits"

sentences = split_into_sentences(caption)


In [7]:
#hold those separate sentences in a dictionary
dict={}
for i in range (len(sentences)):
    dict[sentences.index(sentences[i])] = sentences[i]

In [8]:
#Compute embeddings for our caption

embeddings2={}
for i in range(len(dict)):
    embeddings2[i] = model.encode(dict[i], convert_to_tensor=True)

In [9]:
#Compute similarity scores (caption versus health claims dataset)
scores={}
scores=scores.fromkeys(range(len(dict)))

for i in range (len(dict)):
    scores[i]={}

for i in range (len(scores)):
    for j in range (len(sentences1)):
        scores[i][j] = util.pytorch_cos_sim(embeddings1[j], embeddings2[i])


In [10]:
# check the maximum similarity scores to see if there is one exceeding 0.45

sentences_scores=[]

for i in range(len(scores)):
    temp=[]
    for j in range (len(scores[i])):
        # print("{:.4f}".format(scores[i][j][0][0]))
        temp.append(scores[i][j].item())
    sentences_scores.append(max(temp))
    
for i in range (len(sentences_scores)):
    if sentences_scores[i]>0.45:
        print ("sentence: ", dict[i], " - Score: ", sentences_scores[i])


sentence:  regular cannabis use can greatly contribute to stopping addictive habits.  - Score:  0.601939857006073
